In [5]:
import pandas as pd
import dask.dataframe as dd
from gensim.models import Word2Vec
docs = dd.read_parquet("../output.pq/")

In [8]:
texts = docs['News_Tokens'].compute()

In [ ]:
model = Word2Vec(sentences=texts, vector_size=100, window=5,min_count=1, sg=0)
model.save('word2vec_sent_5_sgu.model')

Exception in thread Thread-62 (_worker_loop):
Traceback (most recent call last):
  File "C:\Users\vadim\scoop\apps\python\3.12.6\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-61 (_worker_loop):
Traceback (most recent call last):
  File "C:\Users\vadim\scoop\apps\python\3.12.6\Lib\threading.py", line 1075, in _bootstrap_inner
Exception in thread Thread-60 (_worker_loop):
Traceback (most recent call last):
  File "C:\Users\vadim\scoop\apps\python\3.12.6\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\vadim\Desktop\6 sem\Курсовая\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    self.run()
  File "c:\Users\vadim\Desktop\6 sem\Курсовая\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    self.run()
  File "c:\Users\vadim\Desktop\6 sem\Курсовая\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\vadim\scoop\